In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/plenoi/CMU_DataScience/master/lung.csv?fbclid=IwAR0nveqa0HXedDudevJF0eAfuvsoGRpmxi7bhGNiz9vBoF9ucYCdO2eiSLs',header=None)
df

In [ ]:
df_clean = df
y = df_clean[0].values
X = df_clean.drop([0],axis = 1).values

from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=0)
X, y = sm.fit_resample(X, y)

from sklearn.model_selection import train_test_split
X, Xt, y, yt = train_test_split(X, y, test_size=0.2, random_state=0)

from sklearn.decomposition import PCA
fr = PCA(n_components=80)
fr.fit(X)
X = fr.transform(X)
Xt = fr.transform(Xt)


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X)
X_train_norm = scaler.transform(X)
X_test_norm = scaler.transform(Xt)


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
params = {
    'C' : [1,2,4,8,16,32], # High C = Overfitting
    'gamma' : [0.01, 0.1,  1, 10 ,100] # High gamma = Overfitting
}
clf = GridSearchCV(SVC(),params, cv=10)
clf.fit(X_train_norm, y)
print("Best params : " + str(clf.best_params_))
print("10CV accuracy : "+str(clf.best_score_*100))

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_predict = clf.predict(X_test_norm)
target_names = ['negative', 'positive']
sum(yt == y_predict)/len(yt)

In [ ]:
df_t = pd.read_csv('https://raw.githubusercontent.com/plenoi/CMU_DataScience/master/lung_test.csv?fbclid=IwAR2A04oyAg8RRe1y-AqJtG0wPZFt1frSZmTZKFx7_xYPbLs5DUfaYmlnMRo',header=None)
df_test_Clean = df_t

In [ ]:
y_test_data = df_test_Clean[0].values
X_test_data = df_test_Clean.drop([0],axis = 1).values
X_test_data = fr.transform(X_test_data)
X_test_data_norm = scaler.transform(X_test_data)

In [ ]:
y_predict = clf.predict(X_test_data_norm)
print("Test accuracy : "+str(sum(y_test_data == y_predict)/len(y_test_data)*100))